In [1]:
!pip install --quiet arcgis

In [2]:
from arcgis.gis import GIS;
from arcgis.features import FeatureLayer,Table;
import requests;

In [23]:
huc12_url = 'https://watersgeo.epa.gov/arcgis/rest/services/Support/HydrologicUnits/MapServer';
huc12_lyr = FeatureLayer(huc12_url + '/6');

huc12s = huc12_lyr.query(
    where = '1=1',
    out_fields = 'huc12',
    return_geometry = False,
    return_all_records = False,
    result_record_count = 1000,
    result_offset = 3000
);

huc12s.sdf.count()

OBJECTID    1000
huc12       1000
dtype: int64

In [ ]:
attains_url = 'https://attains.epa.gov/attains-public/api/huc12summary';
agol_url    = 'https://services1.arcgis.com/BN5Mj6q47j71sqyJ/arcgis/rest/services/ATTAINS_Assessments/FeatureServer/0/query';

for index, row in huc12s.sdf.iterrows():
    response = requests.get(attains_url,params={'huc': row.huc12});
    resp_json = response.json();
    assessments = resp_json["items"][0]["assessmentUnits"];
    
    if len(assessments) > 0:
       qry = 'assessmentunitidentifier%20in%20(';
       for item in assessments:
            qry = qry + '%27' + item["assessmentUnitId"] + '%27%2C%20'
            
       qry = qry.rstrip('%2C%20') + ")"
    
       bam_url = agol_url + '?f=pbf&outFields=*&spatialRel=esriSpatialRelIntersects&where=' + qry;
       response1 = requests.get(bam_url + '&ondev=1');
       
       if response1.status_code != 200:
           raise RuntimeError(response1.request.url);
    
       print(row.huc12 + " " + str(len(assessments)) + " Assessments - " + response1.headers["x-cache"]);
       response2 = requests.get(bam_url + '&onstg=1');
       response3 = requests.get(bam_url);
        
    else:
       print(row.huc12 + " No Assessments")
    

020403030204 2 Assessments - Hit from cloudfront
020403030205 5 Assessments - Hit from cloudfront
020403030206 5 Assessments - Hit from cloudfront
020403030301 8 Assessments - Hit from cloudfront
020403030302 9 Assessments - Hit from cloudfront
020403030303 8 Assessments - Hit from cloudfront
020403030304 8 Assessments - Hit from cloudfront
020403030401 9 Assessments - Hit from cloudfront
020403030402 8 Assessments - Hit from cloudfront
020403030403 6 Assessments - Hit from cloudfront
020403030501 2 Assessments - Miss from cloudfront
020403030502 4 Assessments - Miss from cloudfront
020403030503 No Assessments
020403030504 No Assessments
020403030601 1 Assessments - Miss from cloudfront
020403030602 No Assessments
020403030603 1 Assessments - Miss from cloudfront
020403030604 No Assessments
020403030605 No Assessments
020403040101 No Assessments
020403040102 No Assessments
020403040103 No Assessments
020403040104 No Assessments
020403040201 No Assessments
020403040202 No Assessments
02